In [1]:
from calcula_resultados import compute_indicators_from_files, load_balancetes_to_df, extract_accounts
import pandas as pd
import json
files = ["balancetes/balancete1.json"]
result = compute_indicators_from_files(files)

In [2]:
with open("balancetes/nfeat/Balancete.2023-01.normalized.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [11]:
# ---- DataFrame por arquivo ----
df_balancete = pd.json_normalize(
    result["per_file"]).drop(columns=["file", "n_leaves"])


df_balancete

,indicators.receita_bruta,indicators.impostos_vendas,indicators.receita_liquida,indicators.custos,indicators.lucro_bruto,indicators.despesas,indicators.lucro_liquido,indicators.disponibilidade_caixa
0,684563.19,3160.22,681402.97,405696.63,275706.34,416239.67,-140533.33,130876.58


In [3]:
def extract_accounts(node, hierarchy=None):
    if hierarchy is None:
        hierarchy = []

    current_hierarchy = hierarchy + [node["descricao"]]
    rows = []

    if "children" in node:
        for child in node["children"]:
            rows.extend(extract_accounts(child, current_hierarchy))
    else:
        row = {f"nivel_{i+1}": level for i,
               level in enumerate(current_hierarchy)}
        row["conta"] = node["conta"]
        row["descricao"] = node["descricao"]
        row["saldo_atual"] = node.get("saldo_atual", 0.0)
        rows.append(row)

    return rows

In [4]:
all_rows = []
for section in data.values():
    if isinstance(section, dict) and "descricao" in section:
        all_rows.extend(extract_accounts(section))

# Create a DataFrame
df = pd.DataFrame(all_rows)

In [5]:
df

,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,conta,descricao,saldo_atual
0,ATIVO,ATIVO CIRCULANTE,DISPONIBILIDADES,CAIXA,Caixa,01.1.1.01.001,Caixa,101546.33
1,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,CLIENTES NACIONAIS,Clientes nacionais,01.1.2.01.001,Clientes nacionais,39090.00
2,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,TRIBUTOS E CONTRIBUIÇÕES A COMPENSAR,ICMS a Recuperar,01.1.2.08.001,ICMS a Recuperar,1113.79
3,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,ESTOQUES,Estoque de Mercadoria Para Revenda,01.1.2.10.001,Estoque de Mercadoria Para Revenda,13968.70
4,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,REMESSA DE PRODUTOS/MERCADORIAS,Remessa de Produtos/Mercadorias,01.1.2.12.001,Remessa de Produtos/Mercadorias,13500.00
5,ATIVO,ATIVO NÃO CIRCULANTE,IMOBILIZADO,BENS E DIREITOS EM USO,Máquinas e Equipamentos,01.2.3.01.001,Máquinas e Equipamentos,18024.51
6,ATIVO,ATIVO NÃO CIRCULANTE,IMOBILIZADO,(-) DEPRECIAÇÃO ACUMULADA,(-) Máquinas e equipamentos,01.2.3.05.001,(-) Máquinas e equipamentos,-185.20
7,PASSIVO,PASSIVO CIRCULANTE,FORNECEDORES,REZENDE CENTRO COMERCIAL AUTOMOTIVO LTDA,NaN,02.1.1.01.000007895,REZENDE CENTRO COMERCIAL AUTOMOTIVO LTDA,2434.00
8,PASSIVO,PASSIVO CIRCULANTE,FORNECEDORES,VPMB MARCAS E PATENTES LTDA,NaN,02.1.1.01.001803522,VPMB MARCAS E PATENTES LTDA,4050.32
9,PASSIVO,PASSIVO CIRCULANTE,FORNECEDORES,Comercial Artmaq Ltda EPP,NaN,02.1.1.01.001804167,Comercial Artmaq Ltda EPP,2610.00


In [10]:

df[df["nivel_1"].str.startswith("A")]

,nivel_1,nivel_2,nivel_3,nivel_4,conta,descricao,saldo_atual


In [61]:
with open('balancetes/Balancetes23year_Industria.json', 'r', encoding='utf-8') as f:
    dados_json = json.load(f)